In [307]:
import pandas as pd
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
import os
from tqdm import tqdm
import lyricsgenius

In [308]:
cid = '65760ed785144654a236af484ebad583'
secret = '48ba7ca6ab6548cda428d92a521941af'

In [309]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=30)

In [310]:
def artistas_genero(genre, offset=1, popularity_umbral=15, size=None):

    # Request de search endpoint query=genre, tipo playlist.
    response = spotify.search(q=genre, type='playlist', market="AR", limit=None, offset=offset)

    #Extraigo el id de las playlists que devuelve response
    playlists = [playlist['id'] for playlist in response['playlists']['items']]

    #Creo una lista vacia que se convertirá en el df final y un set de artistas para comprobar repeticiones
    artists_data = []
    seen_artist_ids = set()

    # While loop para obtener la cantidad de artistas especificada en size

    while size is None or len(artists_data) < size:
        #itero sobre cada playlist ID
        for playlist_id in playlists:
            results = spotify.playlist_tracks(playlist_id)
            tracks = results['items']

            # Itero sobre todos los tracks de la playlist
            for track in tracks:
                track_info = track['track']

                # Itero sobre los artistas de cada track
                for artist in track_info['artists']:
                    artist_id = artist['id']

                    # Compruebo repeticiones
                    if artist_id not in seen_artist_ids:
                        seen_artist_ids.add(artist_id)


                        # Intento obtener más información del artista
                        try:
                            artist_data = spotify.artist(artist_id)
                        except:
                            artist_data = None

                        # Compruebo si el artista produce rock nacional
                        if artist_data is not None and ("argentine rock" in artist_data["genres"] or "rock nacional" in artist_data["genres"]):

                            #Filtro los artistas por un umbral de popularidad
                            if artist_data['popularity'] > popularity_umbral:

                                # Extraigo el nombre, id, genre y popularidad del artista.
                                artist_info = {
                                    'Artist_Name': artist['name'],
                                    'Artist_ID': artist_id,
                                    "Artist_genres": artist_data['genres'],
                                    "Artist_popularity": artist_data['popularity']
                                }
                                artists_data.append(artist_info)

                    if size is not None and len(artists_data) >= size:
                        break

                if size is not None and len(artists_data) >= size:
                    break

            if size is not None and len(artists_data) >= size:
                break
    return pd.DataFrame(artists_data)


all_artists_argentina = artistas_genero("Rock Nacional", size=5)
all_artists_argentina

,Artist_Name,Artist_ID,Artist_genres,Artist_popularity
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69
1,Attaque 77,5rI6C5mJm6GYXbGHhpHTu9,"[argentine punk, argentine rock, rock en espan...",55
2,Catupecu Machu,059ysTnWcrm4yFwTr0NDjc,"[argentine punk, argentine rock]",51
3,Andrés Calamaro,3tAICgiSR5PfYY4B8qsoAU,"[argentine rock, cantautor, latin alternative,...",68
4,Los Tipitos,1SykQGBiBwkQ1fcGpJ1BJt,[argentine rock],50


In [311]:
def get_songs(artist_name, n=10):
    #Obtengo n tracks para artista
    results = spotify.search(q=f"artist:{artist_name}",  type='track', offset=0, limit=n)

    #Extraigo canción, id de canción, release y popularidad.
    songs = [result["name"] for result in results["tracks"]["items"]]
    songs_id = [result["id"] for result in results["tracks"]["items"]]
    songs_release = [result["album"]["release_date"] for result in results["tracks"]["items"]]
    songs_popularity = [result["popularity"] for result in results["tracks"]["items"]]

    return songs, songs_id, songs_release,songs_popularity

all_artists_argentina[["song","song_id","song_release","song_popularity"]] = all_artists_argentina["Artist_Name"].apply(lambda x: get_songs(x)).apply(pd.Series)
all_artists_argentina = all_artists_argentina.explode(["song","song_id","song_release","song_popularity"])
all_artists_argentina

,Artist_Name,Artist_ID,Artist_genres,Artist_popularity,song,song_id,song_release,song_popularity
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Crimen,3oqWr0jDWNXxWufNogGREp,2006-04-04,76
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Puente,6gwaa6ElIixNTvu6RwkMyo,1999-06-01,72
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Adiós,1rh4kDY9T4fHVDum8Foi5k,2006-04-04,74
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Lago en el Cielo,17eJyYLIlMNlOqcwHYJ9F2,2006-04-04,69
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Bocanada,12uaDRCVrgu4O6AyOZLrxG,1999-06-01,64
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Deja Vu,6WgUNHiMwJazeG4kWPAH1m,2009-03-12,70
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Te Llevo para Que Me Lleves,5rGphVLnbPdyBQ5X3rjb2X,1993-11-01,65
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Cactus,1NeZxBuSEhikl0ZrOSgpuW,2009-03-12,68
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Vivo,1ShyiEm2kD0XMA7fNhMIVk,2002-11-11,63
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Beautiful,1R3mCl9ya7FanRNWTeiQUE,1999-06-01,60


In [312]:
def get_audio_features(song_id):

    #Extraigo los audio features para cada canción
    features = spotify.audio_features(song_id)
    try:
        features = {k: features[0][k] for k in list(features[0])[:11]}
    except:
        {}
    return features

all_artists_argentina["song_features_dict"] = all_artists_argentina["song_id"].apply(lambda x: get_audio_features(x))

In [313]:
all_artists_argentina

,Artist_Name,Artist_ID,Artist_genres,Artist_popularity,song,song_id,song_release,song_popularity,song_features_dict
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Crimen,3oqWr0jDWNXxWufNogGREp,2006-04-04,76,"{'danceability': 0.537, 'energy': 0.502, 'key'..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Puente,6gwaa6ElIixNTvu6RwkMyo,1999-06-01,72,"{'danceability': 0.444, 'energy': 0.471, 'key'..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Adiós,1rh4kDY9T4fHVDum8Foi5k,2006-04-04,74,"{'danceability': 0.495, 'energy': 0.596, 'key'..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Lago en el Cielo,17eJyYLIlMNlOqcwHYJ9F2,2006-04-04,69,"{'danceability': 0.539, 'energy': 0.833, 'key'..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Bocanada,12uaDRCVrgu4O6AyOZLrxG,1999-06-01,64,"{'danceability': 0.47, 'energy': 0.575, 'key':..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Deja Vu,6WgUNHiMwJazeG4kWPAH1m,2009-03-12,70,"{'danceability': 0.491, 'energy': 0.862, 'key'..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Te Llevo para Que Me Lleves,5rGphVLnbPdyBQ5X3rjb2X,1993-11-01,65,"{'danceability': 0.646, 'energy': 0.596, 'key'..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Cactus,1NeZxBuSEhikl0ZrOSgpuW,2009-03-12,68,"{'danceability': 0.665, 'energy': 0.411, 'key'..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Vivo,1ShyiEm2kD0XMA7fNhMIVk,2002-11-11,63,"{'danceability': 0.453, 'energy': 0.477, 'key'..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Beautiful,1R3mCl9ya7FanRNWTeiQUE,1999-06-01,60,"{'danceability': 0.685, 'energy': 0.684, 'key'..."


In [316]:
def get_lyrics(song_title, artist_name, API_key="FOGZaOj1ZsJYbN7cnaqLtg4wzaTYN-0Yckrq3XyXcykO7rkXB6ZSIDwy7-LQ6Lhm"):
    
    genius = lyricsgenius.Genius(API_key)

    try:
        song = genius.search_song(song_title, artist_name)
        if song:
            return song.lyrics
        else:
            return ""
    except Exception as e:
        print(f"Error occurred: {e}")
        return ""
all_artists_argentina["Lyrics"] = all_artists_argentina.apply(lambda row: get_lyrics(row["song"], row["Artist_Name"]), axis=1)

In [315]:
all_artists_argentina

,Artist_Name,Artist_ID,Artist_genres,Artist_popularity,song,song_id,song_release,song_popularity,song_features_dict,Lyrics
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Crimen,3oqWr0jDWNXxWufNogGREp,2006-04-04,76,"{'danceability': 0.537, 'energy': 0.502, 'key'...","21 ContributorsCrimen Lyrics[Letra de ""Crimen""..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Puente,6gwaa6ElIixNTvu6RwkMyo,1999-06-01,72,"{'danceability': 0.444, 'energy': 0.471, 'key'...",22 ContributorsTranslationsEnglishPuente Lyric...
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Adiós,1rh4kDY9T4fHVDum8Foi5k,2006-04-04,74,"{'danceability': 0.495, 'energy': 0.596, 'key'...","16 ContributorsAdiós Lyrics[Letra de ""Adiós""]\..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Lago en el Cielo,17eJyYLIlMNlOqcwHYJ9F2,2006-04-04,69,"{'danceability': 0.539, 'energy': 0.833, 'key'...",15 ContributorsLago en el Cielo Lyrics[Letra d...
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Bocanada,12uaDRCVrgu4O6AyOZLrxG,1999-06-01,64,"{'danceability': 0.47, 'energy': 0.575, 'key':...","15 ContributorsBocanada Lyrics[Letra de ""Bocan..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Deja Vu,6WgUNHiMwJazeG4kWPAH1m,2009-03-12,70,"{'danceability': 0.491, 'energy': 0.862, 'key'...","8 ContributorsDéjà Vu Lyrics[Letra de ""Déjà Vu..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Te Llevo para Que Me Lleves,5rGphVLnbPdyBQ5X3rjb2X,1993-11-01,65,"{'danceability': 0.646, 'energy': 0.596, 'key'...",10 ContributorsTe Llevo Para Que Me Lleves Lyr...
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Cactus,1NeZxBuSEhikl0ZrOSgpuW,2009-03-12,68,"{'danceability': 0.665, 'energy': 0.411, 'key'...","5 ContributorsCactus Lyrics[Letra de ""Cactus""]..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Vivo,1ShyiEm2kD0XMA7fNhMIVk,2002-11-11,63,"{'danceability': 0.453, 'energy': 0.477, 'key'...","11 ContributorsVivo Lyrics[Letra de ""Vivo""]\n\..."
0,Gustavo Cerati,1QOmebWGB6FdFtW7Bo3F0W,"[argentine rock, latin alternative, latin rock...",69,Beautiful,1R3mCl9ya7FanRNWTeiQUE,1999-06-01,60,"{'danceability': 0.685, 'energy': 0.684, 'key'...","10 ContributorsBeautiful Lyrics[Letra de ""Beau..."
